In [ ]:
!pip install gensim &> /dev/null
!pip install -U pip setuptools wheel &> /dev/null
!pip install texthero &> /dev/null
!pip install -U spacy &> /dev/null
!pip install simplemma &> /dev/null

In [ ]:
import string
import io
import os
import numpy as np
import pandas as pd
import re
#import texthero
from re import sub
import nltk
from tqdm import tqdm
import simplemma
import gensim
import pickle
from gensim.models import Word2Vec
from tqdm import tqdm
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import csv
import scipy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from tqdm import tqdm


nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
STOP_WORDS = set(
    """
а автентичен аз ако ала
бе без беше би бивш бивша бившо бивши бил била били било благодаря близо бъдат
бъде бъда бяха
в вас ваш ваша вашата вашият вероятно вече взема ви вие винаги внимава време все 
всеки всички вместо всичко вследствие всъщност всяка втори във въпреки върху
вътре веднъж 
г ги главен главна главно глас го годно година години годишен
д да дали далеч далече два двама двамата две двете ден днес дни до добра добре 
добро добър достатъчно докато докога дори досега доста друг друга другаде други
е евтин едва един една еднаква еднакви еднакъв едно екип ето
живот жив
за здравей здрасти знае зная забавям зад зададени заедно заради засега заспал 
затова запазва започвам защо защото завинаги
и из или им има имат иска искам използвайки изглежда изглеждаше изглеждайки 
извън имайки
й йо 
каза казва казвайки казвам как каква какво както какъв като кога кауза каузи 
когато когото което които кой който колко която къде където към край кратък 
кръгъл
лесен лесно ли летя летиш летим лош
м май малко макар малцина междувременно минус ме между мек мен месец ми мис 
мисля много мнозина мога могат може мой можем мокър моля момента му
н на над назад най наш навсякъде навътре нагоре направи напред надолу наистина 
например наопаки наполовина напоследък нека независимо нас насам наскоро 
настрана необходимо него негов нещо нея ни ние никой нито нищо но нов някак нова 
нови новина някои някой някога някъде няколко няма
о обаче около описан опитах опитва опитвайки опитвам определен определено освен 
обикновено осигурява обратно означава особен особено от ох отвъд отгоре отдолу 
отново отива отивам отидох отсега отделно отколкото откъдето очевидно оттам 
относно още
п пак по повече повечето под поне просто пряко поради после последен последно 
посочен почти прави прав прави правя пред преди през при пък първата първи първо 
път пъти плюс
равен равна различен различни разумен разумно
с р са сам само себе сериозно сигурен сигурно се сега си син скоро скорошен след 
следващ следващия следва следното следователно случва сме смях собствен 
сравнително смея според сред става срещу съвсем съдържа съдържащ съжалявам 
съответен съответно сте съм със също
т така техен техни такива такъв твърде там трета твой те тези ти то това 
тогава този той търси толкова точно три трябва тук тъй тя тях
у утре ужасно употреба успоредно уточнен уточняване
харесва харесали хиляди
ч часа ценя цяло цялостен че често чрез чудя
ще щеше щом щяха
юмрук
я як
""".split()
)

In [ ]:
from tqdm import tqdm

corpus = []
path = 'folds'
processed_files = 0

for root, dirs, files in os.walk(path):
    for file in tqdm(files):
        if file.endswith('.txt'):
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as f:
                text = f.read()
                sentences = nltk.sent_tokenize(text)
                corpus.extend(sentences)
            processed_files += 1

with open('sentences.txt', 'w') as f:
    f.write('\n'.join(corpus))

0it [00:00, ?it/s]
100%|██████████| 25256/25256 [14:05<00:00, 29.86it/s] 


In [ ]:
corpus_bg = []
with open(r'sentences.txt') as file:
    for line in file:
        corpus_bg.append(line.rstrip())

In [ ]:
corpus_bg[:20]

['',
 'ПРИЛОЖЕНИЕ I',
 '',
 'КРАТКА ХАРАКТЕРИСТИКА НА ПРОДУКТА',
 '',
 '1 1.',
 'ИМЕ НА ЛЕКАРСТВЕНИЯ ПРОДУКТ',
 '',
 'Fendrix инжекционна суспензия.',
 'Ваксина срещу хепатит B (р- ДНК) (Адсорбирана, с адювант) [ Hepatitis B (rDNA) vaccine (adjuvanted, adsorbed) ].',
 '2.',
 'КАЧЕСТВЕН И КОЛИЧЕСТВЕН СЪСТАВ',
 '',
 'Една доза (0, 5 ml) Fendrix съдържа:',
 '',
 'Хепатит B повърхностен антиген 1, 2, 3 1 адювантен с AS04C съдържащ:',
 '',
 '20 микрограма',
 '',
 '- 3- O- дезацил- 4 ’ - монофосфорил липид A (MPL) 2 2 адсорбиран върху алуминиев фосфат (общо 0, 5 милиграма Al3+) 3']

In [ ]:
def preprocess_text(text_list, stop_words):
    # Remove elements with <> in them
    text_list = [x for x in text_list if "<" not in x and ">" not in x]
    
    # Make all elements lowercase
    text_list = [x.lower() for x in text_list]
    
    # Get rid of punctuation
    table = str.maketrans('', '', string.punctuation)
    text_list = [x.translate(table) for x in text_list]
    
    # Get rid of double spaces
    text_list = [re.sub(' +', ' ', x) for x in text_list]
    
    # Get rid of stopwords
    text_list = [x for x in text_list if x not in STOP_WORDS]

    text_list = [re.sub(r'\d+', '', x) for x in text_list]
    
    return text_list

preprocessed_text = preprocess_text(corpus_bg, STOP_WORDS)
preprocessed_text[:20]

['',
 'приложение i',
 '',
 'кратка характеристика на продукта',
 '',
 ' ',
 'име на лекарствения продукт',
 '',
 'fendrix инжекционна суспензия',
 'ваксина срещу хепатит b р днк адсорбирана с адювант hepatitis b rdna vaccine adjuvanted adsorbed ',
 '',
 'качествен и количествен състав',
 '',
 'една доза   ml fendrix съдържа',
 '',
 'хепатит b повърхностен антиген     адювантен с asc съдържащ',
 '',
 ' микрограма',
 '',
 '  o дезацил  ’ монофосфорил липид a mpl   адсорбиран върху алуминиев фосфат общо   милиграма al ']

In [ ]:
lemmatized_text = []
romanian_numbers = ["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"]

for line in preprocessed_text:
    words = line.split()
    stemmed_line = []
    for word in words:
        normalized_word = simplemma.lemmatize(word, lang='bg')
        stemmed_line.append(normalized_word)
    lemmatized_text.append(" ".join(stemmed_line))

lemmatized_text = [s.replace("»", "").replace("«", "") for s in lemmatized_text if s.strip()]

# Regular expression pattern to match English letters
english_pattern = re.compile(r'[a-zA-Z]')

lemmatized_text = [re.sub(r'\b\d+\b', '', string) for string in lemmatized_text if string not in romanian_numbers]
lemmatized_text = [string for string in lemmatized_text if not english_pattern.search(string)]

# Print the lemmatized text
print(lemmatized_text[:20])

['кратък характеристика на продукт', 'име на лекарствен продукт', 'качествен и количествен състав', 'микрограм', 'микрограм', 'за пълен списък на помощните вещество вж', 'точка', 'лекарствен форма', 'инжекционна суспензия', 'мътен бял суспензия', 'при съхраня мога да се наблюдавам образуване на фин бял утайка с бистър безцветен надутаечен слой', 'клинични данни', 'терапевтични покажа', 'дозировка и начин на приложение', 'дозировка', 'схема за първичен ваксинация', 'препоръчвам се схема с прилагане на четири доза като имунизирането съм на избран дата месец месец и месец след прилагане на първи доза', 'бустер доза', 'начин на приложение', 'противопокажа']


In [ ]:
def save_list_to_csv(list_, file_name):
    with open(file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        for row in list_:
            writer.writerow([row])
save_list_to_csv(lemmatized_text, 'stemmed_text_bg_new.csv')

In [ ]:
stem_corp= pd.read_csv('stemmed_text_bg_new.csv',header=None, names=['text'])

In [ ]:
stem_corp = stem_corp[stem_corp['text'].str.split().str.len() >= 4]

In [ ]:
stem_corp

,text
0,кратък характеристика на продукт
1,име на лекарствен продукт
2,качествен и количествен състав
5,за пълен списък на помощните вещество вж
10,при съхраня мога да се наблюдавам образуване н...
...,...
1003848,Македония и Албания ща съм сътруднича в гранич...
1003849,организиран с подкрепа на нато инициатива ща п...
1003850,ръководител на отдел на нато за балкан робърт ...
1003851,върховен представител пади ашдаун заявя пред р...


In [ ]:
vectorizer = CountVectorizer(min_df=5)

data_vectorized = vectorizer.fit_transform(stem_corp['text'][5000::10])

In [ ]:
A = data_vectorized.toarray()
A = A.astype(float)

In [ ]:
feature_names = sorted(vectorizer.vocabulary_.keys())

In [ ]:
len(feature_names)

11540

In [ ]:
A.shape

(79470, 11540)

In [ ]:
K = 10
u, s, vt = scipy.sparse.linalg.svds(A.T, k=K)

In [ ]:
A.T.shape

(11540, 79470)

In [ ]:
u.shape, s.shape, vt.shape

((11540, 10), (10,), (10, 79470))

In [ ]:
with open('bg_vt_' + str(K) +  '.pickle', 'wb') as f:
  pickle.dump(vt, f)
with open('bg_u_' + str(K) + '.pickle', 'wb') as f:
  pickle.dump(u, f)
with open('bg_s_.' + str(K) + 'pickle', 'wb') as f:
  pickle.dump(s, f)

In [ ]:
print(feature_names[20:50])

['абдулах', 'абк', 'абнормен', 'абнормна', 'абнормни', 'абнормно', 'абонат', 'аборт', 'аборти', 'абортите', 'абсолютен', 'абсолютно', 'абсорбирам', 'абсорбция', 'абстиненция', 'абсурден', 'абсцес', 'ав', 'аваз', 'август', 'авив', 'авиокомпания', 'авиолиния', 'авоарите', 'австралийски', 'австралия', 'австриец', 'австрийски', 'австрия', 'австроунгарски']


In [ ]:
!pip install langdetect >> None

In [ ]:
import string
from langdetect import detect

def preprocess_text(text):
    # Detect language of the text
    lang = detect(text)
    # Remove punctuation and convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words and words in English and Greek
    if lang == 'bg':
        stop_words = STOP_WORDS
    else:
        stop_words = STOP_WORDS.union(set(word_tokenize(' '.join(stopwords.words(lang)))))

    tokens = [token for token in tokens if token not in stop_words and not any(char in string.ascii_letters for char in token)]
    # Perform morphological analysis and lemmatization
    lemmatized_tokens = []
    for token in tokens:
        lemma = simplemma.lemmatize(token, lang=lang)
        lemmatized_tokens.append(lemma)
    # Join the preprocessed tokens back into a single string
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

def convertTextToEmbeddingSeries(text, dict_words, emb):
    wtext = preprocess_text(text)
    wdata = wtext.split(' ')
    word2ind = dict(zip(dict_words, np.arange(len(dict_words))))
    embText = []
    for word in wdata:
        ind = word2ind.get(word)
        if ind: embText.append(emb[ind])
    return np.array(embText)

In [ ]:
preprocessed_text = preprocess_text('организиран с подкрепа на нато инициатива ща')
preprocessed_text

'организирам подкрепа нато инициатива ща'

In [ ]:
textSeries = convertTextToEmbeddingSeries(preprocessed_text,
                                          feature_names,
                                          u)

In [ ]:
textSeries.shape

(5, 10)

In [ ]:
textSeries

array([[-5.05386215e-04,  2.52836735e-04,  1.88880391e-04,
         1.29095354e-03,  1.86181262e-03,  3.27577801e-04,
         2.06188861e-04,  1.17232112e-03, -6.37433036e-04,
        -1.10048655e-03],
       [ 1.04532651e-03, -2.60634074e-03,  1.65438937e-04,
         4.31073947e-03,  8.98719318e-03, -9.17829061e-04,
         5.68939917e-03,  1.13627826e-02, -9.88337586e-04,
        -6.59270644e-03],
       [ 1.85782742e-04, -8.44670433e-03, -1.66386780e-03,
         9.98367580e-03,  1.19292952e-02, -4.29382379e-03,
         5.91022888e-03,  7.83156737e-03, -2.90202188e-04,
        -8.55594902e-03],
       [-2.36933377e-03,  7.27782687e-04,  6.92567807e-04,
         1.54191652e-03,  1.97946816e-03,  3.27364123e-04,
         1.22075155e-03,  2.13434742e-03, -3.08917544e-04,
        -1.84925066e-03],
       [-1.00441914e-02, -2.58634779e-01, -2.85811231e-02,
         1.02404900e-01,  1.06312855e-01,  2.08094046e-02,
         1.95244851e-02,  4.23311675e-02, -2.72297058e-02,
        -4.